<div style="background-color: #fde70e; padding: 10px; display: flex; align-items: center; border-bottom: 2px solid #fde70e;">
  <a href="https://www.fhnw.ch/en/about-fhnw/schools/school-of-engineering/institutes/institute-of-electric-power-systems" target="_blank">
    <img src="./Logo_FHNW.png" alt="FHNW Logo" style="height: 40px; margin-right: 15px;">
  </a>
</div>

# Berechnungstool für den Energieverbrauch von Antriebssystemen
Dieses Tool berechnet den Energieverbrauch eines Antriebssystems, basierend auf einem Lastprofil und Wirkungsgrad-Daten.

<details>
  <summary style="font-size: 1.1em; font-weight: bold; cursor: pointer;">📂 Anleitung zum Hochladen von Daten-Files</summary>
  <div style="margin-top: 10px;">
    <p>Lade Excel-Dateien mit den folgenden Inhalten hoch und gib die nötigen Parameter ein:</p>
    <ol>
      <li><strong>Betriebsprofil des Antriebssystems:</strong> In Spalte A unter dem Titel "Power" werden die zeitlichen Werte der Ausgangsleistung angegeben. Jede Zeile steht für einen Zeitschritt. Grösse und Einheit der Zeitschritte werden bei den Eingabeparametern angegeben.</li>
      <li><strong>Wirkungsgraddaten des betrachteten Systems:</strong> In Spalte A unter dem Titel "Power" diejenigen Leistungspunkte, an denen Wirkungsgradwerte angegeben werden. In Spalte B unter dem Titel "Efficiency" die zugehörigen Wirkungsgradwerte (0...1).</li>
      <li><strong>Optional:</strong> Wenn ein Energieverbrauchs-Vergleich von zwei verschiedenen Systemen erstellt werden soll, kann ein zweites Excel-Dokument mit weiteren Wirkungsgraddaten hochgeladen werden.</li>
    </ol>
      <div style="display: flex; justify-content: center; gap: 40px; margin-top: 10px;">

  <div style="text-align: center;">
    <img src="./exceltemplate_eff_p.png" alt="Excel-Vorlage Wirkungsgrad" style="width: auto; height: 250px;">
    <p><em>Abb. 1: Wirkungsgradtabelle</em></p>
  </div>

  <div style="text-align: center;">
    <img src="./exceltemplate_up_power.png" alt="Excel-Vorlage Betriebsprofil" style="width: auto; height: 250px;">
    <p><em>Abb. 2: Betriebsprofil-Tabelle</em></p>
  </div>

</div>
  </div>
</details>

<details>
  <summary style="font-size: 1.1em; font-weight: bold; cursor: pointer;">⚙️ Anleitung zur Parameter-Eingabe</summary>
  <div style="margin-top: 10px;">
    <ul>
      <li><strong>Max. Power:</strong> Maximale Ausgangsleistung des Systems. Dieser Wert darf nicht grösser sein als der höchste Leistungswert in der Wirkungsgradtabelle. Angabe in Watt.</li>
      <li><strong>Price:</strong> Strompreis in CHF/kWh zur Berechnung der Energiekosten.</li>
      <li><strong>Timestep:</strong> Grösse eines Zeitschritts in der Excel-Datei mit dem Betriebsprofil.</li>
      <li><strong>Time unit:</strong> Einheit der Zeitschritte (Sekunden / Minuten / Stunden)</li>
      <li><strong>Fit:</strong> Für die beiden hochgeladenen Wirkungsgrad-Daten können unterschiedliche Fit-Funktionen gewählt werden, um zwischen den Datenpunkten zu interpolieren. Die Datenpunkte und die Fitfunktionen werden angezeigt, wenn der User auf „Berechnen“ klickt.</li>
    </ul>
  </div>
</details>

<details>
  <summary style="font-size: 1.1em; font-weight: bold; cursor: pointer;">📊 Ausgaben des Tools</summary>
  <div style="margin-top: 10px;">
    <p>Es werden die folgenden Ausgaben generiert:</p>
    <ul>
      <li>Darstellung der Wirkungsgraddaten und der Fitfunktionen</li>
      <li>Energieverbrauch, Nutzenergie, Verluste und Energiekosten für einen Durchlauf des Betriebsprofils</li>
      <li>Energieverbrauch, Nutzenergie, Verluste und Energiekosten hochgerechnet auf 1 Jahr</li>
      <li>Zeitliche Verläufe der Eingangs- und Ausgangsleistung des Systems</li>
    </ul>
  </div>
</details>

<div style="background-color: #fde70e; border-left: 5px solid #f0c000; padding: 15px 20px; margin-top: 30px; box-shadow: 2px 2px 5px rgba(0,0,0,0.1);">
  <h2 style="margin: 0;">🛠️ Eingabebereich des Tools</h2>
  <p style="margin: 5px 0 0;">Hier beginnt der interaktive Teil: Eingabe von Parametern, Auswahl von Dateien und Start der Berechnung.</p>
</div>



In [ ]:
import pandas as pd
import numpy as np
import io
from scipy.interpolate import interp1d, UnivariateSpline
from numpy.polynomial.polynomial import Polynomial
from ipywidgets import FileUpload, FloatText, Dropdown, Button, VBox, HBox, Label, Output, Layout, Checkbox
from IPython.display import display
import matplotlib.pyplot as plt

# Eingabefelder
max_power_field = FloatText(value=1000.0, layout=Layout(width='150px'))
energy_price_field = FloatText(value=0.20, layout=Layout(width='150px'))
timestep_field = FloatText(value=1.0, layout=Layout(width='150px'))
energy_unit_field = Dropdown(
    options=['seconds', 'minutes', 'hours'],
    value='seconds',
    layout=Layout(width='150px')
)

# Fit-Auswahl
fit_type_field = Dropdown(
    options=[
        ("Linear", "linear"),
        ("Polynom Grad 2", "poly2"),
        ("Polynom Grad 3", "poly3"),
        ("Spline", "spline"),
        ("Stückweise linear", "interp")
    ],
    value="interp",
    description="Fit:"
)

# Upload-Widgets
upload_usage_profile = FileUpload(accept='.xlsx', multiple=False)
upload_efficiency_data = FileUpload(accept='.xlsx', multiple=False)

# Checkbox zur optionalen Verwendung einer zweiten Effizienzkurve
use_second_eff_field = Checkbox(
    value=False,
    description="Zweite Effizienzkurve hochladen & vergleichen"
)

# 2. Upload-Feld für Effizienzdatei 2
upload_efficiency_data_2 = FileUpload(accept='.xlsx', multiple=False)

# Zweites Dropdown für Fitfunktion 2
fit_type_field_2 = Dropdown(
    options=[
        ("Linear", "linear"),
        ("Polynom Grad 2", "poly2"),
        ("Polynom Grad 3", "poly3"),
        ("Spline", "spline"),
        ("Stückweise linear", "interp")
    ],
    value="interp",
    description="Fit 2:"
)

# Layouts
max_power_input = HBox([max_power_field, Label("Max Power [W]")])
energy_price_input = HBox([energy_price_field, Label("Price [CHF/kWh]")])
timestep_input = HBox([timestep_field, Label("Timestep")])
energy_unit_dropdown = HBox([energy_unit_field, Label("Time unit")])

uploader_section = VBox([
    Label("📂 Excel-Dateien hochladen:"),
    VBox([
        Label("🔼 Lade das Nutzungsprofil hoch (.xlsx mit Spalte 'Power')"),
        upload_usage_profile,
        Label("Format: Eine Spalte 'Power' in Watt")
    ]),
    VBox([
        Label("🔼 Lade die Effizienzkurve 1 hoch (.xlsx mit Spalten 'Power' & 'Efficiency')"),
        upload_efficiency_data,
        Label("Format: Zwei Spalten 'Power' und 'Efficiency'"),
        fit_type_field  # <- Fit 1 Dropdown bleibt hier
    ]),
    use_second_eff_field,
    VBox([
        Label("🔼 Lade die Effizienzkurve 2 hoch (.xlsx mit Spalten 'Power' & 'Efficiency')"),
        upload_efficiency_data_2,
        Label("Format: Zwei Spalten 'Power' und 'Efficiency'"),
        fit_type_field_2
    ])
])

# Zusammenfassung der Eingabefelder
input_section = VBox([
    Label("🧮 Parameter eingeben:"),
    max_power_input,
    energy_price_input,
    timestep_input,
    energy_unit_dropdown
])

# Button & Output
calculate_button = Button(description='Berechnen', button_style='success')
output = Output()

# Daten-Container
usage_profile_df = pd.DataFrame()
efficiency_df = pd.DataFrame()

# Energieintegration
def Integrate(vec, timestep, unit='seconds'):
    if unit == 'minutes':
        t = timestep * 60
    elif unit == 'hours':
        t = timestep * 3600
    else:
        t = timestep
    return sum([v * t for v in vec]) * 2.77777e-7

# Fit-Erstellung
def fit_efficiency_curve(df, method):
    x = df['Power'].values
    y = df['Efficiency'].values

    if method == "linear":
        coeffs = np.polyfit(x, y, 1)
        return lambda p: np.polyval(coeffs, p)
    elif method == "poly2":
        coeffs = np.polyfit(x, y, 2)
        return lambda p: np.polyval(coeffs, p)
    elif method == "poly3":
        coeffs = np.polyfit(x, y, 3)
        return lambda p: np.polyval(coeffs, p)
    elif method == "spline":
        spline = UnivariateSpline(x, y, s=0)
        return spline
    else:
        return interp1d(x, y, bounds_error=False, fill_value="extrapolate")

# Fit-Plot
def plot_efficiency_fit(df, method):
    x = df['Power'].values
    y = df['Efficiency'].values
    fit_fn = fit_efficiency_curve(df, method)

    x_plot = np.linspace(min(x), max(x), 300)
    y_plot = fit_fn(x_plot)

    # Optional: R² berechnen
    r_squared = None
    if method in ["linear", "poly2", "poly3"]:
        y_fit = fit_fn(x)
        ss_res = np.sum((y - y_fit) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r_squared = 1 - (ss_res / ss_tot)

    plt.figure(figsize=(8, 4))
    plt.plot(x, y, 'o', label="Originaldaten")
    plt.plot(x_plot, y_plot, '-', label=f"Fit: {fit_type_field.label}")

    if r_squared is not None:
        plt.text(0.05, 0.95, f"R² = {r_squared:.4f}",
                 transform=plt.gca().transAxes,
                 fontsize=10,
                 verticalalignment='top',
                 bbox=dict(boxstyle="round", facecolor="white", edgecolor="gray"))

    plt.xlabel("Power [W]")
    plt.ylabel("Efficiency")
    plt.title("Effizienzkurve mit Fit")
    plt.grid(True)
    plt.legend()
    plt.show()

# Leistungsplot
def plot_power_curves(power_out, power_in1, timestep, unit, power_in2=None):
    if unit == 'minutes':
        dt = timestep * 60
    elif unit == 'hours':
        dt = timestep * 3600
    else:
        dt = timestep

    t = [i * dt for i in range(len(power_out))]

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(t, power_out, label='P_out (Lastprofil)', linewidth=2)
    ax.plot(t, power_in1, label='P_in 1 (Effizienz 1)', linewidth=2, linestyle='--')

    if power_in2 is not None:
        ax.plot(t, power_in2, label='P_in 2 (Effizienz 2)', linewidth=2, linestyle=':', color='green')

    ax.set_xlabel('Zeit [s]')
    ax.set_ylabel('Leistung [W]')
    ax.set_title('Verlauf von P_out, P_in 1 und ggf. P_in 2')
    ax.grid(True)
    ax.legend()
    plt.show()

# Funktion compare_results(...)
def compare_results(ein1, ecost1, edelta1, ein2, ecost2, edelta2):
    df_compare = pd.DataFrame({
        "Grösse": ["Energieverbrauch [kWh]", "Energiekosten [CHF]", "Energieverlust [kWh]"],
        "Effizienz 1": [f"{ein1:.3f}", f"{ecost1:.2f}", f"{edelta1:.3f}"],
        "Effizienz 2": [f"{ein2:.3f}", f"{ecost2:.2f}", f"{edelta2:.3f}"]
    })
    display(df_compare)

# Vergleich jährlicher Daten
def compare_yearly_results(ein1, ecost1, edelta1, ein2, ecost2, edelta2, duration_s):
    seconds_per_year = 365 * 24 * 3600
    cycles_per_year = seconds_per_year / duration_s

    ein1_y = ein1 * cycles_per_year
    ecost1_y = ecost1 * cycles_per_year
    edelta1_y = edelta1 * cycles_per_year

    ein2_y = ein2 * cycles_per_year
    ecost2_y = ecost2 * cycles_per_year
    edelta2_y = edelta2 * cycles_per_year

    df_year = pd.DataFrame({
        "Grösse (Jahreswerte)": ["Energieverbrauch [kWh]", "Energiekosten [CHF]", "Energieverlust [kWh]"],
        "Effizienz 1": [f"{ein1_y:.0f}", f"{ecost1_y:.0f}", f"{edelta1_y:.0f}"],
        "Effizienz 2": [f"{ein2_y:.0f}", f"{ecost2_y:.0f}", f"{edelta2_y:.0f}"]
    })

    print("\n📆 Hochrechnung auf 1 Jahr:")
    display(df_year)

# Hauptfunktion
def handle_calculate(b):
    global usage_profile_df, efficiency_df
    output.clear_output()
    
    power_in2 = None  # vorbereiten für optionalen zweiten Plot

    if not upload_usage_profile.value or not upload_efficiency_data.value:
        with output:
            print("⚠️ Bitte Excel-Dateien für Profil und Effizienz 1 hochladen.")
        return

    try:
        usage_data = upload_usage_profile.value[0]
        efficiency_data = upload_efficiency_data.value[0]

        usage_name = usage_data['name']
        eff_name = efficiency_data['name']

        usage_profile_df = pd.read_excel(io.BytesIO(usage_data['content']))
        efficiency_df = pd.read_excel(io.BytesIO(efficiency_data['content']))

        power_out = usage_profile_df['Power'].tolist()

        # Fit 1 + Berechnung
        efficiency = fit_efficiency_curve(efficiency_df, fit_type_field.value)
        power_in = [p / efficiency(p) for p in power_out]
        ein = Integrate(power_in, timestep_field.value, unit=energy_unit_field.value)
        eout = Integrate(power_out, timestep_field.value, unit=energy_unit_field.value)
        edelta = ein - eout
        ecost = ein * energy_price_field.value

        with output:
            print(f"✅ Verwendete Dateien:")
            print(f"📂 Usage Profile: {usage_name}")
            print(f"📂 Efficiency Data 1: {eff_name}")

            # Effizienz-Plot 1
            plot_efficiency_fit(efficiency_df, fit_type_field.value)

            print("🔋 Energieverbrauch (P_in 1): {:.3f} kWh".format(ein))
            print("⚙️ Nutzenergie (P_out): {:.3f} kWh".format(eout))
            print("🔥 Energieverlust: {:.3f} kWh".format(edelta))
            print("💰 Energiekosten: {:.2f} CHF".format(ecost))

            # 🔁 Dauer des Profils (in Sekunden) für Hochrechnung
            timestep = timestep_field.value
            unit = energy_unit_field.value
            duration_s = len(power_out) * timestep
            if unit == 'minutes':
                duration_s *= 60
            elif unit == 'hours':
                duration_s *= 3600

            cycles_per_year = (365 * 24 * 3600) / duration_s
            ein_year = ein * cycles_per_year
            edelta_year = edelta * cycles_per_year
            ecost_year = ecost * cycles_per_year

            print("\n📆 Hochrechnung auf 1 Jahr (Effizienz 1):")
            print("🔋 Jahresverbrauch: {:.0f} kWh".format(ein_year))
            print("🔥 Jahresverlust: {:.0f} kWh".format(edelta_year))
            print("💰 Jahreskosten: {:.0f} CHF".format(ecost_year))

            if use_second_eff_field.value and upload_efficiency_data_2.value:
                try:
                    eff2_data = upload_efficiency_data_2.value[0]
                    eff2_name = eff2_data['name']
                    eff2_df = pd.read_excel(io.BytesIO(eff2_data['content']))

                    print(f"\n📂 Efficiency Data 2: {eff2_name}")
                    plot_efficiency_fit(eff2_df, fit_type_field_2.value)

                    efficiency2 = fit_efficiency_curve(eff2_df, fit_type_field_2.value)
                    power_in2 = [p / efficiency2(p) for p in power_out]

                    ein2 = Integrate(power_in2, timestep_field.value, unit=energy_unit_field.value)
                    edelta2 = ein2 - eout
                    ecost2 = ein2 * energy_price_field.value

                    print("🔋 Energieverbrauch (P_in 2): {:.3f} kWh".format(ein2))
                    print("🔥 Energieverlust: {:.3f} kWh".format(edelta2))
                    print("💰 Energiekosten: {:.2f} CHF".format(ecost2))

                    print("\n📊 Vergleich der Verbrauchsdaten im Betriebsprofil für die beiden Effizienzkurven:")
                    compare_results(ein, ecost, edelta, ein2, ecost2, edelta2)

                    # Jahreswerte berechnen
                    timestep = timestep_field.value
                    unit = energy_unit_field.value
                    duration_s = len(power_out) * timestep
                    if unit == 'minutes':
                        duration_s *= 60
                    elif unit == 'hours':
                        duration_s *= 3600

                    compare_yearly_results(ein, ecost, edelta, ein2, ecost2, edelta2, duration_s)

                except Exception as e2:
                    print("❌ Fehler bei zweiter Effizienzberechnung:", e2)

            # Plot aller Leistungen (P_out, P_in 1, ggf. P_in 2)
            plot_power_curves(
                power_out,
                power_in,
                timestep_field.value,
                energy_unit_field.value,
                power_in2
            )

    except Exception as e:
        with output:
            print("❌ Fehler bei der Berechnung:", e)

# Klick verbinden
calculate_button.on_click(handle_calculate)

# GUI anzeigen
app = VBox([
    uploader_section,
    input_section,
    calculate_button,
    output
])

display(app)


In [ ]:
import matplotlib.pyplot as plt

def plot_power_curves(power_out, power_in1, timestep, unit, power_in2=None):
    if unit == 'minutes':
        dt = timestep * 60
    elif unit == 'hours':
        dt = timestep * 3600
    else:
        dt = timestep

    t = [i * dt for i in range(len(power_out))]

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(t, power_out, label='P_out (Lastprofil)', linewidth=2)
    ax.plot(t, power_in1, label='P_in 1 (Effizienz 1)', linewidth=2, linestyle='--')

    if power_in2 is not None:
        ax.plot(t, power_in2, label='P_in 2 (Effizienz 2)', linewidth=2, linestyle=':', color='green')

    ax.set_xlabel('Zeit [s]')
    ax.set_ylabel('Leistung [W]')
    ax.set_title('Verlauf von P_out, P_in 1 und ggf. P_in 2')
    ax.grid(True)
    ax.legend()
    plt.show()
